# Тестовое задание для команды "Анализ эмоциональной окраски постов клиентов банка в условиях кризиса"

## Импорт необходимых библиотек

In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression 

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
from tqdm import notebook

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/balyadavid/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/balyadavid/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [1]:
import torch
import transformers 

Загругка RuBert быда сделана в этом месте, потому что по какой-то причине, если сделать это после предобработки данных, ядро крашится.

In [2]:
config = transformers.BertConfig.from_json_file(
     'rubert1/bert_config.json')
tokenizer = transformers.BertTokenizer(
    vocab_file='rubert1/vocab.txt') 


In [3]:
model = transformers.BertModel.from_pretrained(
    'rubert1/pytorch_model.bin', config=config)

Some weights of the model checkpoint at rubert1/pytorch_model.bin were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Изучение данных из файла

Было решено использовать датасет с новостями ресурса лента.ру, потому что не я нашел необходимый с комментариями для классификации (нашел только с разметкой токсичных/не токсичных), в данном датесете представлены новости с разметкой по теме новости.

In [6]:
df = pd.read_csv('lenta-ru-news.csv')

In [7]:
df.head(15)

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31


Посмотрим на уникальные значения тегов новостей:

In [8]:
df['tags'].unique()

array(['Первая мировая', 'Все', nan, 'Прибалтика', 'Кино', 'Преступность',
       'Общество', 'Происшествия', 'Искусство', 'Бизнес', 'Техника',
       'ТВ и радио', 'Политика', 'Пресса', 'Музыка', 'Люди', 'Звери',
       'Игры', 'Госэкономика', 'Гаджеты', 'Наука', 'Еда', 'Рынки',
       'Деньги', 'Летние виды', 'Интернет', 'Театр', 'Конфликты',
       'Реклама', 'Космос', 'Бокс и ММА', 'Футбол', 'Книги',
       'Зимние виды', 'Достижения', 'Coцсети', 'Вещи', 'События',
       'Средняя Азия', 'Украина', 'Закавказье', 'Белоруссия', 'Молдавия',
       'Софт', 'Квартира', 'Город', 'Дача', 'Офис', 'Оружие', 'Мнения',
       'Москва', 'Регионы', 'Полиция и спецслужбы', 'Криминал',
       'Следствие и суд', 'Движение', 'Производители', 'Мировой бизнес',
       'Финансы компаний', 'Деловой климат', 'Мир', 'Россия', 'Часы',
       'Явления', 'Стиль', 'Инструменты', 'Вооружение', 'Вкусы',
       'Страноведение', 'Госрегулирование', 'История', 'Внешний вид',
       'Автобизнес', 'Аналитика рынка'

Посмотрим на уникальные значения тем новостей:

In [9]:
df['topic'].unique()

array(['Библиотека', 'Россия', 'Мир', 'Экономика', 'Интернет и СМИ',
       'Спорт', 'Культура', 'Из жизни', 'Силовые структуры',
       'Наука и техника', 'Бывший СССР', nan, 'Дом', 'Сочи', 'ЧМ-2014',
       'Путешествия', 'Ценности', 'Легпром', 'Бизнес', 'МедНовости',
       'Оружие', '69-я параллель', 'Культпросвет ', 'Крым'], dtype=object)

Для классификации новостей по темам нам не нужны дата и url новости, поэтому сразу дропаю их.

In [10]:
df = df.drop(['url', 'date'], axis=1)

Был изучен процент пропусков в столбцах, и расположена информация в порядке убывания:

In [11]:
df.isna().mean().sort_values(ascending = False)

topic    0.077408
tags     0.033982
text     0.000006
title    0.000000
dtype: float64

Было решено удалить все строки датасета с пропусками, так как это будет мешать дальнейшей работе с дангными и классификации в целом.

In [12]:
df = df.dropna().reset_index(drop = True)

Проверка, все ли работает исправно после удаления:

In [13]:
df.head()

,title,text,topic,tags
0,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая
1,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая
2,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая
3,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая
4,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая


Было решено вывести проверить процент тем новостей по мере убывания, чтобы в дальнейшем исключить темы с наиболее низкими процентами встречаемости, для того, чтобы модели было проще учиться:

In [14]:
df['topic'].value_counts(normalize=True)

Россия               0.217502
Мир                  0.191706
Экономика            0.107237
Спорт                0.081237
Культура             0.075114
Наука и техника      0.074561
Бывший СССР          0.072083
Интернет и СМИ       0.062332
Из жизни             0.038606
Дом                  0.030497
Силовые структуры    0.015748
Ценности             0.010638
Бизнес               0.010349
Путешествия          0.008938
69-я параллель       0.001779
Крым                 0.000935
Культпросвет         0.000477
Легпром              0.000160
Библиотека           0.000091
Оружие               0.000004
ЧМ-2014              0.000003
МедНовости           0.000001
Сочи                 0.000001
Name: topic, dtype: float64

In [15]:
less_one_persent = ['Путешествия', '69-я параллель', 'Крым', 'Культпросвет ', 'Легпром', 'Библиотека', 'Оружие', 'ЧМ-2014', 'МедНовости', 'Сочи']

In [16]:
df = df[~df['topic'].isin(less_one_persent)]

Проверка данных:

In [18]:
df['topic'].value_counts(normalize=True)

Россия               0.220231
Мир                  0.194111
Экономика            0.108583
Спорт                0.082256
Культура             0.076056
Наука и техника      0.075496
Бывший СССР          0.072987
Интернет и СМИ       0.063114
Из жизни             0.039091
Дом                  0.030880
Силовые структуры    0.015946
Ценности             0.010771
Бизнес               0.010478
Name: topic, dtype: float64

Также былы проверены данные на дубликаты и удалены найденные:

In [19]:
df.duplicated().sum()

17

In [20]:
df = df.drop_duplicates()

Проверка:

In [21]:
df.duplicated().sum()

0

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703807 entries, 5 to 712653
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   703807 non-null  object
 1   text    703807 non-null  object
 2   topic   703807 non-null  object
 3   tags    703807 non-null  object
dtypes: object(4)
memory usage: 26.8+ MB


Была сделана настройка отображения датасета, для того, чтобы видеть общую картину:

In [25]:
pd.set_option('display.max_colwidth', 0)
warnings.filterwarnings("ignore")

## Работа с RuBert

Было решено взять только часть данных, для того, чтобы модель работала быстрее:

In [51]:
df_bert = df.sample(20000, random_state=244466666).reset_index(drop=True)

In [52]:
df_bert.head()

,title,text,topic,tags
0,Sukhoi Superjet 100 аварийно сел в Подмосковье,"Российский ближнемагистральный пассажирский самолет Sukhoi Superjet 100 совершил аварийную посадку в Летно-исследовательском институте имени Михаила Громова в подмосковном Раменском, сообщил РИА Новости генеральный директор института Евгений Пушкарский. «Факт такой имеется. Все завершилось нормально. [У самолета] была соответствующая неисправность», — сказал руководитель. По словам Пушкарского, никто не пострадал, а полеты на территории института продолжаются в штатном режиме. В феврале с Sukhoi Superjet 100 произошло сразу семь инцидентов. Первый российский гражданский самолет Sukhoi Superjet 100 вмещает в базовой версии 93-102 пассажира. В 2011-2017 годах выпущено 156 самолетов. Крупнейшим покупателем Sukhoi Superjet 100 стал «Аэрофлот» (35 самолетов), второе место приходится на мексиканскую Interjet.",Наука и техника,Техника
1,Нулевую ставку экспортной пошлины на нефть оставят до весны,"Нулевая ставка пошлины на экспорт нефти из Восточной Сибири сохранится как минимум до марта. Об этом заявил заместитель министра финансов России Сергей Шаталов, сообщает РИА Новости. Решение о повышении ставок будет принято отдельно, добавил Шаталов. Ранее сообщалось, что министр финансов Алексей Кудрин предложил восстановить экспортную пошлину на восточносибирскую нефть, нулевая ставка которой начала действовать с января 2010 года. Он заявил, что из-за ""скоропалительного и непросчитанного"" решения об обнулении пошлины бюджет потеряет в 2010 году около 80 миллиардов рублей. Вместе с тем, как пишет газета ""Время новостей"", льготный режим экспорта нефти фактически уже не действует. Дело в том, что с 1 января в России в рамках создания единого экономического пространства с Белоруссией и Казахстаном начала действовать новая таможенная номенклатура. Российской нефти, вывозимой за рубеж, пришлось сменить код, тогда как льготы действовали для продукции под прежним кодом. Чтобы его изменить, необходимо внести поправки в постановление правительства. Ранее вице-премьер Игорь Сечин предлагал распространить нулевую ставку пошлины на восточносибирскую нефть только на экспорт по трубопроводу ВСТО. Таким образом правительство рассчитывало заполнить стратегически важный нефтепровод и ограничить экспорт в западном направлении.",Экономика,Все
2,Херсонская область установила противотанковые ежи на границе с Крымом,"Губернатор Херсонской области Андрей Гордеев заявил, что на границе с Крымом в регионе началась установка заградительных конструкций и противотанковых ежей. Об этом в пятницу, 23 сентября, он сказал в интервью телеканалу «112 Украина». «Мы вместе с Минтрансом строим противотанковые ежи. Устанавливаем эти конструктивные элементы, согласно дислокации, определенной Генштабом», — заявил Гордеев. Он добавил, что в укреплении границы участвует также министерство обороны и Херсонский областной совет. О подготовке к установлению заграждений в Херсонской области рассказал 26 августа первый заместитель председателя комитета Рады по вопросам науки и образования Александр Спиваковский. По его словам, в регионе существует угроза вторжения танковых и моторизованных подразделений российской армии. «Я знаю, что Херсонщина получила семь тысяч метров металлических рельсов, швеллеров для изготовления ежей. Они будут установлены, чтобы не было разных неожиданностей», — подчеркнул Спиваковский. 18 августа украинский президент Петр Порошенко заявил, что не исключает полномасштабного российского вторжения. Представитель департамента Главного управления разведки Минобороны Украины Вадим Скибицкий пояснил, что наступление российские войска могут начать из Крыма. По его словам, Россия сосредоточила на севере полуострова мощную группировку. В ответ представитель Министерства обороны США Джефф Дэвис отметил, что Пентагон не считает передвижения российских войск в Крыму и у границ Украины признаком готовящегося вторжения. По его словам, перемещение войск связано только с учениями российской армии. Киев

In [53]:
df_bert.tail()

,title,text,topic,tags
19995,Заказавшего свою жену певца христианского металла отпустили под залог,"Вокалиста христианской металкор группы As I Lay Dying Тима Ламбезиса, пытавшегося организовать убийство своей бывшей жены, отпустили из-под ареста под залог в 2 миллиона долларов. Об этом сообщает NBC News. Неизвестно, выплатил ли музыкант сумму целиком или использовал залоговое поручительство. Ламбезис был задержан днем 7 мая в торговом центре города Оушенсайд, в Калифорнии. В момент задержания он пытался нанять человека для убийства проживающей отдельно от него жены — Мегган Ламбезис. По данным следователей, певец спросил некоего человека в тренажерном зале, не знает ли тот, кого-нибудь, кто мог бы убить его жену. Мотивом Ламбезиса, как выяснила полиция, было решение Мегган отобрать у музыканта троих приемных детей. В сентябре 2012 года она подала документы для бракоразводного процесса в окружной суд. Ламбезис решил, что для детей будет лучше, если они будут постоянно жить только с одним из родителей. Группа As I Lay Dying существует с 2000 года. Последний альбом коллектива — «Awakened» — вышел в сентябре 2012 года. Группа активно гастролирует -накануне ареста Ламбезис вернулся в США из Китая, где был с концертами.",Мир,Преступность
19996,Канье Уэст собрался шить детскую одежду,"Американский рэпер Канье Уэст и муж Ким Кардашьян решил создать линейку одежды для детей. Как сообщает WWD, именно эта идея будет лежать в основе его новой коллекции — Yeezy Season 4. Какие именно модели войдут в линейку и когда она будет представлена, артист не уточнил. Карьеру дизайнера Уэст начал в 2015 году с сотрудничества с adidas Originals. Он выпустил уже три коллекции одежды, вторая была представлена осенью. В работе над шоу участвовала итальянская художница Ванесса Бикрофт (Vanessa Beecroft), которая использует обнаженную натуру для перформансов. Публике показали куртки, шорты, обтягивающие лосины, а также платья и купальники. Третья по счету коллекция была презентована в начале февраля в рамках Недели моды в Нью-Йорке и совпала с днем релиза нового альбома музыканта — Swish. В линейку вошли несколько новых моделей худи и свитеров камуфляжного цвета. Цены варьируются от 545 до 2600 долларов. Канье Уэст начал выступать в начале 2000-х годов. На счету артиста более 21 миллиона проданных дисков и 66 миллионов легально скачанных альбомов. Певец удостоился 21 статуэтки «Грэмми» и был номинирован на эту награду 53 раза.",Ценности,Явления
19997,Пассажир своими руками обезвредил драчунью в утреннем метро,"Пассажир метро обезвредил и удерживал до приезда полиции женщину, которая устроила драку в переполненном поезде. Об этом сообщает The New York Post. Инцидент произошел в метро Нью-Йорка около восьми утра. 40-летняя Анна Лущинская бросилась на близстоящую девушку, выкрикивая расистские оскорбления. Люди пытались оттащить драчунью, но она не прекращала проявлять агрессию. Видеоролики с произошедшим опубликовал пользователь Twitter под ником @PlatanoMan. «Когда она полезла ко мне, пришлось ее обезвредить до приезда копов», — говорится в еще одном твите американца. Он также сопроводил публикацию двумя снимками, на которых держит Лущинскую и передает ее полиции. Портал heavy отмечает, что это далеко не первый случай, когда женщина попадается на драке: на ее счету около десятка задержаний. Например, ранее ее задерживали за то, что она плевалась в прохожих.",Из жизни,Происшествия
19998,24 тысячи индонезийцев приготовились к эвакуации с подножья вулкана,"Индонезийские власти готовятся эвакуировать около 24 тысяч человек, проживающих у вулкана Келуд на острове Ява. Ученые предупредили о вероятности сильного извержения, передает агентство AP. В последние дни специалисты зафиксировали повышение сейсмической активности в районе вулкана, а также увеличение температуры и изменение химического состава воды в озере, которое находится в кратере. Вулкан Келуд высотой 1731 метр входит в число самых активных вулканов Индонезии. В последний раз он извергался в 1990 году

### Предодбработка текста

Инициализировал токенизатор в самом начале ноутбука, так как иначе все крашилось

Токенизатор нужен для того, чтобы разбить и преобразовать исходные тексты в список токенов, которые есть в словаре RuBert, при работе с RuBert лемматизация не требуется

In [54]:
tokenized = df_bert['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

In [55]:
tokenized

0        [101, 13605, 1524, 11466, 852, 82013, 36127, 46624, 852, 15577, 11466, 852, 16674, 10710, 49014, 15150, 25724, 19045, 7993, 15939, 18918, 3065, 26838, 845, 3422, 1438, 130, 56364, 16720, 7778, 26476, 12515, 3297, 19668, 3735, 845, 50909, 82513, 33519, 128, 9880, 66385, 626, 28338, 9456, 896, 1468, 852, 7997, 12141, 5933, 14788, 1524, 43380, 46272, 2237, 852, 132, 304, 9728, 2306, 15484, 15625, 132, 4752, 41815, 43880, 132, 222, 875, 17284, 226, 3370, 56593, 69467, 326, 128, 901, 10615, 15523, 132, 1516, 6335, 43380, 39432, 128, 16963, 1699, 21735, 128, 625, 36003, 1469, 5735, 12141, 35755, 845, 80209, 18641, 132, ...]           
1        [101, 31271, 18834, 39198, 47384, 1469, 17622, 15284, 1703, 32685, 852, 29119, 852, 88713, 2739, 17875, 2785, 5842, 132, 1650, 3491, 6309, 13353, 11160, 9469, 13605, 1524, 3649, 6625, 852, 40336, 7475, 128, 4721, 66385, 626, 28338, 132, 8253, 612, 44012, 49048, 5022, 13060, 25044, 128, 15472, 40336, 7475, 132, 8403, 13940, 128, 1997, 10709, 94

Далее был применен метод padding, чтобы после токенизации длины исходных текстов в корпусе были равными. Только при таком условии будет работать модель BERT.  

In [56]:
padded = np.array([i + [0]*(512 - len(i)) for i in tokenized.values])

In [57]:
padded.shape[0]

20000

Attention_mask нужен для того, чтобы "объяснить" модели, что нули не несут информации

In [58]:
attention_mask = np.where(padded != 0, 1, 0)

In [59]:
attention_mask.shape

(20000, 512)

Далее тексты были преобразованы в эмбединги

In [60]:
%%time
batch_size = 200
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 18h 25min 35s, sys: 5h 1min, total: 23h 26min 35s
Wall time: 2h 44min 9s


In [61]:
embeddings

[array([[-0.14588594, -0.04769892, -0.02683488, ...,  0.12396687,
          0.08633386,  0.05565508],
        [ 0.04112832, -0.08977963, -0.13821808, ..., -0.05122468,
          0.17569055, -0.09114888],
        [-0.3181637 ,  0.32692057, -0.29761276, ...,  0.12454744,
          0.5329617 , -0.46586466],
        ...,
        [ 0.10631215, -0.01709597, -0.02273092, ..., -0.08341794,
          0.13983135, -0.13081084],
        [-0.1650424 ,  0.05131   , -0.04912162, ...,  0.04017805,
          0.14489044, -0.48541117],
        [ 0.28245592,  0.23640572, -0.08781475, ..., -0.05839249,
          0.19386785,  0.09732744]], dtype=float32),
 array([[-0.06510738,  0.17391826, -0.23225865, ...,  0.04409923,
          0.19486539, -0.29496443],
        [-0.24010733,  0.04462022, -0.14234436, ..., -0.0879592 ,
          0.02811623, -0.17192838],
        [-0.37126255,  0.22240673, -0.0581367 , ...,  0.10218243,
         -0.06536274, -0.52160364],
        ...,
        [ 0.2577069 , -0.11014386, -0.0

## Обучение моделей

Была создана таблица для дальнейшего сравнения моделей, также была создана функция для добавления данных в таблицу

In [152]:
scores_of_models = pd.DataFrame(columns=['model_name', 'accuracy_score'])

In [153]:
def scores_of_model(name_of_model, accuracy_score):
    return scores_of_models.append({'model_name' : name_of_model, 'accuracy_score': accuracy_score}, ignore_index=True) 

Из эмбедингов были получены фичи для обучения, также были выбраны таргеты из очищенного датасета

In [62]:
features = np.concatenate(embeddings)
target = df_bert['topic']

In [63]:
features

array([[-0.14588594, -0.04769892, -0.02683488, ...,  0.12396687,
         0.08633386,  0.05565508],
       [ 0.04112832, -0.08977963, -0.13821808, ..., -0.05122468,
         0.17569055, -0.09114888],
       [-0.3181637 ,  0.32692057, -0.29761276, ...,  0.12454744,
         0.5329617 , -0.46586466],
       ...,
       [-0.13988394, -0.125952  , -0.22015366, ..., -0.14544828,
         0.18545066, -0.50079197],
       [-0.15437213, -0.22685465,  0.04521282, ...,  0.17469755,
         0.4085062 , -0.11016873],
       [-0.12159584, -0.20712069, -0.2076453 , ...,  0.21992648,
         0.21285228, -0.2542409 ]], dtype=float32)

In [64]:
test_index = int(len(df_bert)*(1-0.25))
test_index

15000

In [65]:
train_features = features[:test_index]
train_target = target[:test_index]

test_features = features[test_index:]
test_target = target[test_index:]

Непосредственно для обучения были выбраны модели: GaussianNB, LogisticRegression, OneVsOneClassifier, OutputCodeClassifier 

### GaussianNB

In [105]:
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB

In [106]:
Gaussian_model = GaussianNB()
Gaussian_model.fit(train_features, train_target)

GaussianNB()

In [107]:
predictions_Gaussian_model = Gaussian_model.predict(test_features)

In [108]:
from sklearn.metrics import accuracy_score

In [132]:
accuracy_score_Gaussian_model = accuracy_score(test_target, predictions_Gaussian_model)
accuracy_score_Gaussian_model

0.643

In [154]:
scores_of_models = scores_of_model('Gaussian_model', accuracy_score_Gaussian_model)
scores_of_models

,model_name,accuracy_score
0,Gaussian_model,0.643


### LogisticRegression

In [118]:
LogisticRegression_model = LogisticRegression()
LogisticRegression_model.fit(train_features, train_target)
predictions_LogisticRegression_model = LogisticRegression_model.predict(test_features)

In [120]:
accuracy_score_LogisticRegression_model = accuracy_score(test_target, predictions_LogisticRegression_model)
accuracy_score_LogisticRegression_model 

0.7972

In [155]:
scores_of_models = scores_of_model('LogisticRegression', accuracy_score_LogisticRegression_model)
scores_of_models

,model_name,accuracy_score
0,Gaussian_model,0.643
1,LogisticRegression,0.7972


### OneVsOneClassifier

In [138]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

OneVsOneClassifier_model = OneVsOneClassifier(LinearSVC(random_state=0)).fit(train_features, train_target)

In [139]:
predictions_OneVsOneClassifier_model = OneVsOneClassifier_model.predict(test_features)

In [140]:
accuracy_score_OneVsOneClassifier_model = accuracy_score(test_target, predictions_OneVsOneClassifier_model)

In [156]:
scores_of_models = scores_of_model('OneVsOneClassifier', accuracy_score_OneVsOneClassifier_model)
scores_of_models

,model_name,accuracy_score
0,Gaussian_model,0.643
1,LogisticRegression,0.7972
2,OneVsOneClassifier,0.7872


### OutputCodeClassifier

In [142]:
from sklearn import datasets
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC

OutputCodeClassifier_model = OutputCodeClassifier(LinearSVC(random_state=0), random_state=0)

In [143]:
OutputCodeClassifier_model.fit(train_features, train_target)

OutputCodeClassifier(estimator=LinearSVC(random_state=0), random_state=0)

In [147]:
predictions_OutputCodeClassifier_model = OutputCodeClassifier_model.predict(test_features)

In [150]:
accuracy_score_OutputCodeClassifier_model = accuracy_score(test_target, predictions_OutputCodeClassifier)

In [157]:
scores_of_models = scores_of_model('OutputCodeClassifier', accuracy_score_OutputCodeClassifier_model)
scores_of_models

,model_name,accuracy_score
0,Gaussian_model,0.643
1,LogisticRegression,0.7972
2,OneVsOneClassifier,0.7872
3,OutputCodeClassifier,0.7774


## Вывод

В ходе работы были выполнены следующие этапы: 
- Ознакомление с данными
- Предобработка данных
- Работа с RuBert
- Обучение различных моделей для классификации


Самый большой показатель accuracy_score показала логистическая регрессия - 0.7972, в ходе данного исследования она может быть рекомендована для использования в бизнес задачах. Возможнен дальнейший fine-tuning RuBert и подбор гиперпораметров для модели классификации.